In [13]:
import json
import random
import urllib
import numpy as np
import requests
import pandas as pd
import datetime
from copy import deepcopy

from transport_network import Transport_Network,  Polygon_Location, Mode
from activity_scheduler import ActivityScheduler
from mode_choice_nhts import NhtsModeLogit
from two_stage_logit_hlc import TwoStageLogitHLC
from mobility_service_model import MobilityModel
from cs_handler import CS_Handler


In [3]:
tn = Transport_Network('corktown', 'Detroit')

In [4]:
host = 'https://cityio.media.mit.edu/'
table_name = 'corktown'

CITYIO_GET_URL=host+'api/table/'+table_name
with urllib.request.urlopen(CITYIO_GET_URL+'/GEOGRID') as url:
    geogrid_geojson=json.loads(url.read().decode())

In [5]:
this_model = MobilityModel('corktown', 'Detroit')

Building transport network
Building geography
Building synthetic population


In [6]:
this_model.assign_activity_scheduler(ActivityScheduler(model=this_model))

d:\Anaconda\envs\cusplondon2\lib\site-packages\sklearn\base.py:334: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.22.2.post1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
d:\Anaconda\envs\cusplondon2\lib\site-packages\sklearn\base.py:334: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.22.2.post1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [7]:
mode_choice_model=NhtsModeLogit(table_name='corktown', city_folder='Detroit')

In [8]:
this_model.assign_mode_choice_model(mode_choice_model)

In [9]:
this_model.assign_home_location_choice_model(
            TwoStageLogitHLC(table_name='corktown', city_folder='Detroit', 
                             geogrid=this_model.geogrid, base_vacant_houses=this_model.pop.base_vacant))

In [12]:
this_model.init_simulation()

Initialising Simulation
	 Activity Scheduling
	 Creating Trips
	 Predicting Trip modes
	 	 predicting
	 	 applying predictions to trips


In [11]:
this_model.mode_choice_set

AttributeError: 'MobilityModel' object has no attribute 'mode_choice_set'

In [ ]:
print('Initialising Simulation')
print('\t Activity Scheduling')
random.seed(this_model.seed)
np.random.seed(this_model.seed)
this_model.activity_scheduler.assign_profiles(this_model.pop.base_sim)
for person in this_model.pop.base_sim:
    this_model.activity_scheduler.sample_activity_schedules(person, model=this_model)


In [ ]:
this_model.create_trips(this_model.pop.base_sim)
this_model.predict_trip_modes(this_model.pop.base_sim, logit_params=new_logit_params)
this_model.pop.impact=this_model.pop.base_sim
#        this_model.predict_mode_choices()

In [ ]:
class Trip():
    def __init__(self, mode_choice_set, enters_sim, from_activity, to_activity):
        self.enters_sim=enters_sim
        self.activity_start=from_activity.start_time
        self.mode_choice_set=mode_choice_set
        self.mode=None
        if from_activity.activity_id + to_activity.activity_id in ['HW', 'WH']:
            self.purpose='HBW'
        elif 'H' in [from_activity.activity_id , to_activity.activity_id]:
            self.purpose= 'HBO'
        else:
            self.purpose='NHB'
    def set_mode(self, mode, mode_list, utility=None):
#        mode_name=mode_list[mode].name
        copy_route=mode_list[mode].copy_route
        self.mode=mode_list[mode]
        self.internal_route=self.mode_choice_set[copy_route].internal_route['internal_route']
        self.pre_time=self.mode_choice_set[copy_route].pre_time
        self.post_time=self.mode_choice_set[copy_route].post_time
        self.total_distance=self.internal_route['total_distance'
                                ]+((self.pre_time)+(self.post_time))*60*self.mode.speed_met_s
        if utility is not None:
            self.utility=utility

In [ ]:
def create_trips(self, persons):
    print('\t Creating Trips')
    for person in persons:
        trips=[]
        for ind_act in range(len(person.activities)-1):
            origin=person.activities[ind_act]
            destination=person.activities[ind_act+1]
            if not origin==destination:
                mode_choice_set=self.tn.get_routes(origin.location, destination.location)
                enters_sim=((origin.location.area_type=='grid') or (destination.location.area_type=='grid'))
                trips.append(Trip(mode_choice_set, enters_sim=enters_sim,
                                    from_activity=origin, to_activity=destination))
        person.assign_trips(trips)


In [ ]:

def predict_trip_modes(self, persons, method='random', seed=None, logit_params={}):
    print('\t Predicting Trip modes')
    temp_mode_choice_model=deepcopy(self.mode_choice_model)
    all_trips=[]
    for person_id, p in enumerate(persons):
        all_trips.extend(p.trips_to_list(person_id=person_id))
    temp_mode_choice_model.generate_feature_df(all_trips)
    if len(temp_mode_choice_model.new_alt_specs)>0:
        for new_spec in temp_mode_choice_model.new_alt_specs:
            temp_mode_choice_model.set_new_alt(new_spec)
    temp_mode_choice_model.set_logit_model_params(logit_params)
    print('\t \t predicting')
    temp_mode_choice_model.predict_modes(method=method, seed=seed)
    mode_list=self.tn.base_modes+ self.tn.new_modes
    print('\t \t applying predictions to trips')
    for i, trip_record in enumerate(all_trips):
        person_id=trip_record['person_id']
        trip_id=trip_record['trip_id']
        predicted_mode=temp_mode_choice_model.predicted_modes[i]
#            trip_utility=self.mode_choice_model.predicted_v[i][predicted_mode]
        persons[person_id].trips[trip_id].set_mode(predicted_mode, 
                mode_list)

In [ ]:
def init_simulation(self, new_logit_params={}):
    print('Initialising Simulation')
    print('\t Activity Scheduling')
    random.seed(self.seed)
    np.random.seed(self.seed)
    self.activity_scheduler.assign_profiles(self.pop.base_sim)
    for person in self.pop.base_sim:
        self.activity_scheduler.sample_activity_schedules(person, model=self)
    self.create_trips(self.pop.base_sim)
    self.predict_trip_modes(self.pop.base_sim, logit_params=new_logit_params)
    self.pop.impact=self.pop.base_sim
#        self.predict_mode_choices()

In [15]:
handler=CS_Handler(this_model)


Initialising Simulation
	 Activity Scheduling
	 Creating Trips
	 Predicting Trip modes
	 	 predicting
	 	 applying predictions to trips


In [17]:
handler.get_outputs()

{'CO2 Performance raw kg/day': 9.157902319212033,
 'Mobility Health Impacts raw mortality/year': 0.001366596558462891,
 'CO2 Performance norm': 0.40601395439828103,
 'Mobility Health Impacts norm': 0.34164913961572274,
 'driving': 86.58367911479945,
 'cycling': 0.6915629322268326,
 'walking': 6.915629322268327,
 'pt': 5.809128630705394}